<a href="https://colab.research.google.com/github/yhc-666/hate-speech-detection/blob/master/lightgbm_gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import make_scorer
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jul 17 06:03:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
df_dataset = pd.read_csv('/content/drive/MyDrive/50.007kaggle/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/50.007kaggle/test.csv')
df_dataset_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/train_tfidf_features.csv")
df_test_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/test_tfidf_features.csv")
samplesubmit = pd.read_csv("/content/drive/MyDrive/50.007kaggle/sample_submission.csv")

In [6]:
print(samplesubmit.shape)

(4296, 2)


In [7]:
labels = df_dataset_features.iloc[:,[1]]
features = df_dataset_features.iloc[:,2:]
#y_test = df_test.iloc[:,[1]]
#X_test = df_test_features.iloc[:,1:]

#分训练集测试集
X_train, X_validation, y_train, y_validation = train_test_split(features, labels, test_size=0.05, random_state=2)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_validation, y_validation, reference=lgb_train)

In [8]:
print(y_validation)
print(y_train)
print(X_train)
print(X_validation)

       label
7429       1
13425      0
14486      1
13298      0
8879       0
...      ...
8127       0
15828      0
17180      0
14676      0
13342      1

[860 rows x 1 columns]
       label
14525      1
7378       1
7061       1
13158      1
12637      1
...      ...
14696      1
1099       0
11798      0
6637       1
2575       1

[16324 rows x 1 columns]
         0    1    2    3    4    5    6    7    8    9  ...  4990  4991  \
14525  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
7378   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
7061   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
13158  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
12637  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
14696  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1099   0.0  0.0  0.0  0.0  0.0

In [27]:
def my_custom_loss_func(y_true, y_pred):
    pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
    neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
    macro_f1 = (pos_f1 + neg_f1)/2  
    return macro_f1

macro_f1 = make_scorer(my_custom_loss_func, greater_is_better=True)

## Tuning 'max_depth','num_leaves'

In [22]:
mdl = lgb.LGBMClassifier(boosting_type = 'gbdt',  # 设置提升类型
    objective = 'binary',  # 目标函数
    num_iterations = 100,
    # max_depth = -1,
    metric = 'macro_f1',  # 评估函数
    #'num_leaves': 700,  # 叶子节点数
    learning_rate = 0.1,  # 学习速率
    #feature_fraction = 0.9,  # 建树的特征选择比例
    #bagging_fraction = 0.8,  # 建树的样本采样比例
    #bagging_freq =  5,  # k 意味着每 k 次迭代执行bagging
    scale_pos_weight = 1.8,
    #lambda_l2 = 0.01,
    nthread = -1,
    verbose = 1  )


In [34]:
gridParams = {
    #'num_iterations': [50, 100, 200, 500],
    'max_depth' : [50,100,200,500,-1],
    #'learning_rate': [0.005, 0.01, 0.1],
    #'n_estimators': [10, 50, 100, 200, 500],
    'num_leaves': [100, 300, 700, 1000], # large num_leaves helps improve accuracy but might lead to over-fitting
    #"scale_pos_weight" : [1, 2, 3],
    #'lambda_l2': [0, 0.01, 0.1, 1]
    }

In [35]:
grid = GridSearchCV(mdl, gridParams, scoring = 'f1_macro', verbose=1, cv = 3, n_jobs=-1)

In [36]:
grid.fit(X_train,y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it in

GridSearchCV(cv=3,
             estimator=LGBMClassifier(metric='macro_f1', nthread=-1,
                                      num_iterations=100, objective='binary',
                                      scale_pos_weight=1.8, verbose=1),
             n_jobs=-1,
             param_grid={'max_depth': [50, 100, 200, 500, -1],
                         'num_leaves': [100, 300, 700, 1000]},
             scoring='f1_macro', verbose=1)

In [37]:
grid.best_params_, grid.best_score_

({'max_depth': 100, 'num_leaves': 100}, 0.6825187361753248)

In [39]:
mdl = lgb.LGBMClassifier(boosting_type = 'gbdt',  # 设置提升类型
    objective = 'binary',  # 目标函数
    num_iterations = 100,
    # max_depth = -1,
    metric = 'macro_f1',  # 评估函数
    #'num_leaves': 700,  # 叶子节点数
    learning_rate = 0.1,  # 学习速率
    #feature_fraction = 0.9,  # 建树的特征选择比例
    #bagging_fraction = 0.8,  # 建树的样本采样比例
    #bagging_freq =  5,  # k 意味着每 k 次迭代执行bagging
    scale_pos_weight = 1.8,
    #lambda_l2 = 0.01,
    nthread = -1,
    verbose = 1  )


In [38]:
gridParams = {
    #'num_iterations': [50, 100, 200, 500],
    'max_depth' : [75,100,125,150],
    #'learning_rate': [0.005, 0.01, 0.1],
    #'n_estimators': [10, 50, 100, 200, 500],
    'num_leaves': [50, 75, 100, 125, 150], # large num_leaves helps improve accuracy but might lead to over-fitting
    #"scale_pos_weight" : [1, 2, 3],
    #'lambda_l2': [0, 0.01, 0.1, 1]
    }

In [40]:
grid = GridSearchCV(mdl, gridParams, scoring = 'f1_macro', verbose=1, cv = 3, n_jobs=-1)

In [41]:
grid.fit(X_train,y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it in

GridSearchCV(cv=3,
             estimator=LGBMClassifier(metric='macro_f1', nthread=-1,
                                      num_iterations=100, objective='binary',
                                      scale_pos_weight=1.8, verbose=1),
             n_jobs=-1,
             param_grid={'max_depth': [75, 100, 125, 150],
                         'num_leaves': [50, 75, 100, 125, 150]},
             scoring='f1_macro', verbose=1)

In [42]:
grid.best_params_, grid.best_score_

({'max_depth': 75, 'num_leaves': 125}, 0.6837821742648665)

## Tuning n_estimators

In [ ]:
mdl = lgb.LGBMClassifier(boosting_type = 'gbdt',  # 设置提升类型
    objective = 'binary',  # 目标函数
    #num_iterations = 100,
    max_depth = 75,
    metric = 'macro_f1',  # 评估函数
    num_leaves = 125,  # 叶子节点数
    learning_rate = 0.1,  # 学习速率
    #feature_fraction = 0.9,  # 建树的特征选择比例
    #bagging_fraction = 0.8,  # 建树的样本采样比例
    #bagging_freq =  5,  # k 意味着每 k 次迭代执行bagging
    scale_pos_weight = 1.8,
    #lambda_l2 = 0.01,
    nthread = -1,
    verbose = 1  )


In [43]:
gridParams = {
    'num_iterations': [50, 100, 200, 500, 1000],
    #'max_depth' : [50,100,200,500,-1],
    #'learning_rate': [0.005, 0.01, 0.1],
    #'n_estimators': [10, 50, 100, 200, 500],
    #'num_leaves': [100, 300, 700, 1000], # large num_leaves helps improve accuracy but might lead to over-fitting
    #"scale_pos_weight" : [1, 2, 3],
    #'lambda_l2': [0, 0.01, 0.1, 1]
    }

In [44]:
grid = GridSearchCV(mdl, gridParams, scoring = 'f1_macro', verbose=1, cv = 3, n_jobs=-1)

In [45]:
grid.fit(X_train,y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it in

GridSearchCV(cv=3,
             estimator=LGBMClassifier(metric='macro_f1', nthread=-1,
                                      num_iterations=100, objective='binary',
                                      scale_pos_weight=1.8, verbose=1),
             n_jobs=-1,
             param_grid={'num_iterations': [50, 100, 200, 500, 1000]},
             scoring='f1_macro', verbose=1)

In [46]:
grid.best_params_, grid.best_score_

({'num_iterations': 200}, 0.6827132420267691)

## Tuning feature_fraction 和 bagging_fraction (avoid overfitting)

## Tuning lambda_l2 (avoid overfitting)

## Reduce learning rate to improve performance

In [13]:
y_validation, y_pred_validation = y_validation, gbm.predict(X_validation)

threshold = 0.5
for i in range(len(y_pred_validation)):
    y_pred_validation[i] = 1 if y_pred_validation[i] > threshold else 0
print(y_pred_validation)
pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
macro_f1 = (pos_f1 + neg_f1)/2
print(pos_f1)
print(neg_f1)
print("validation_macro_f1 : %.4g" % macro_f1)  
print("validation_acc: %.4g"% metrics.accuracy_score(y_validation, y_pred_validation))

[0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1.
 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1.
 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0.

In [ ]:
X_test = df_test_features.iloc[:,1:]

In [ ]:
print(X_test)

        0    1    2    3    4    5    6    7    8    9  ...  4990  4991  4992  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   
4291  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4292  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4293  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4294  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4295  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   

      4993  4994  4995  499

In [ ]:
y_test = gbm.predict(X_test)

In [ ]:
print(y_test)

[1 1 1 ... 1 0 0]


In [ ]:
x = df_test
x = x.set_index("id")
x = x.drop("post", axis = 1)
x["label"] = y_test
print(x)
x.to_csv("/content/drive/MyDrive/50.007kaggle/submit.csv")

       label
id          
17185      1
17186      1
17187      1
17188      0
17189      0
...      ...
21476      1
21477      1
21478      1
21479      0
21480      0

[4296 rows x 1 columns]
